తదుపరి నోట్బుక్స్ నడపడానికి, మీరు ఇంకా చేయకపోతే, .env ఫైల్‌లో openai కీని `OPENAI_API_KEY`గా సెట్ చేయాలి.


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

తర్వాత, మేము ఎంబెడ్డింగ్ ఇండెక్స్‌ను పాండాస్ డేటాఫ్రేమ్‌లో లోడ్ చేయబోతున్నాము. ఎంబెడ్డింగ్ ఇండెక్స్ `embedding_index_3m.json` అనే JSON ఫైల్‌లో నిల్వ చేయబడింది. ఎంబెడ్డింగ్ ఇండెక్స్‌లో 2023 అక్టోబర్ చివరి వరకు ఉన్న ప్రతి YouTube ట్రాన్స్క్రిప్ట్‌ల కోసం ఎంబెడ్డింగ్స్ ఉంటాయి.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

తర్వాత, మేము `get_videos` అనే ఫంక్షన్‌ను సృష్టించబోతున్నాము, ఇది క్వెరీ కోసం ఎంబెడ్డింగ్ ఇండెక్స్‌ను శోధిస్తుంది. ఈ ఫంక్షన్ క్వెరీకి అత్యంత సమానమైన టాప్ 5 వీడియోలను తిరిగి ఇస్తుంది. ఫంక్షన్ క్రింది విధంగా పనిచేస్తుంది:

1. మొదట, ఎంబెడ్డింగ్ ఇండెక్స్ యొక్క ఒక కాపీ సృష్టించబడుతుంది.
2. తరువాత, OpenAI ఎంబెడ్డింగ్ API ఉపయోగించి క్వెరీ కోసం ఎంబెడ్డింగ్ లెక్కించబడుతుంది.
3. ఆపై, ఎంబెడ్డింగ్ ఇండెక్స్‌లో `similarity` అనే కొత్త కాలమ్ సృష్టించబడుతుంది. `similarity` కాలమ్ క్వెరీ ఎంబెడ్డింగ్ మరియు ప్రతి వీడియో సెగ్మెంట్ కోసం ఎంబెడ్డింగ్ మధ్య కోసైన్ సమానత్వాన్ని కలిగి ఉంటుంది.
4. తరువాత, `similarity` కాలమ్ ఆధారంగా ఎంబెడ్డింగ్ ఇండెక్స్ ఫిల్టర్ చేయబడుతుంది. కోసైన్ సమానత్వం 0.75 లేదా అంతకంటే ఎక్కువ ఉన్న వీడియోలను మాత్రమే ఎంబెడ్డింగ్ ఇండెక్స్‌లో ఉంచబడుతుంది.
5. చివరగా, `similarity` కాలమ్ ఆధారంగా ఎంబెడ్డింగ్ ఇండెక్స్ సార్టు చేయబడుతుంది మరియు టాప్ 5 వీడియోలు తిరిగి ఇవ్వబడతాయి.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

ఈ ఫంక్షన్ చాలా సులభం, ఇది కేవలం శోధన ప్రశ్న యొక్క ఫలితాలను ముద్రిస్తుంది.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. మొదట, ఎంబెడ్డింగ్ సూచికను పాండాస్ డేటాఫ్రేమ్‌లో లోడ్ చేస్తారు.
2. తరువాత, వినియోగదారుని ఒక ప్రశ్నను నమోదు చేయమని అడుగుతారు.
3. ఆపై, `get_videos` ఫంక్షన్‌ను పిలిచి ప్రశ్న కోసం ఎంబెడ్డింగ్ సూచికను శోధిస్తారు.
4. చివరగా, ఫలితాలను వినియోగదారునికి చూపించడానికి `display_results` ఫంక్షన్‌ను పిలుస్తారు.
5. ఆ తర్వాత వినియోగదారుని మరో ప్రశ్నను నమోదు చేయమని అడుగుతారు. వినియోగదారు `exit` అని నమోదు చేసే వరకు ఈ ప్రక్రియ కొనసాగుతుంది.

![](../../../../translated_images/te/notebook-search.1e320b9c7fcbb0bc.png)

మీకు ఒక ప్రశ్నను నమోదు చేయమని అడుగుతారు. ఒక ప్రశ్నను నమోదు చేసి ఎంటర్ నొక్కండి. ఆప్లికేషన్ ఆ ప్రశ్నకు సంబంధించి ఉన్న వీడియోల జాబితాను అందిస్తుంది. ఆప్లికేషన్ ప్రశ్నకు సమాధానం ఉన్న వీడియోలోని స్థానానికి లింక్ కూడా అందిస్తుంది.

ఇక్కడ కొన్ని ప్రయత్నించదగిన ప్రశ్నలు ఉన్నాయి:

- Azure Machine Learning అంటే ఏమిటి?
- కన్‌వల్యూషనల్ న్యూరల్ నెట్‌వర్క్స్ ఎలా పనిచేస్తాయి?
- న్యూరల్ నెట్‌వర్క్ అంటే ఏమిటి?
- నేను Azure Machine Learning తో Jupyter Notebooks ఉపయోగించగలనా?
- ONNX అంటే ఏమిటి?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో పొరపాట్లు లేదా తప్పిదాలు ఉండవచ్చు. అసలు పత్రం దాని స్వదేశీ భాషలోనే అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారానికి, ప్రొఫెషనల్ మానవ అనువాదం సిఫార్సు చేయబడుతుంది. ఈ అనువాదం వాడకంలో ఏర్పడిన ఏవైనా అపార్థాలు లేదా తప్పుదారుల కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
